In [1]:
from pathlib import Path
import pandas as pd
FILE_DIR = Path().resolve().parent
DATA_DIR = FILE_DIR / "2_models"


train_df = pd.read_pickle(f"{DATA_DIR}/train_df.pkl")
val_df = pd.read_pickle(f"{DATA_DIR}/val_df.pkl")

print(f"Train DataFrame shape: {train_df.shape}")
print(f"Validation DataFrame shape: {val_df.shape}")

Train DataFrame shape: (5252, 4)
Validation DataFrame shape: (927, 4)


In [2]:
import random
import numpy as np
import torch
import torch.nn as nn
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(42)

In [3]:
train_df.head()

,length,label,signal,rr_std
1,18000,2,"[-446, -541, -637, -733, -819, -858, -867, -87...",182.922570
3,9000,0,"[-1255, -1488, -1745, -2015, -2253, -2374, -23...",123.855300
4,9000,0,"[156, 189, 223, 255, 291, 330, 362, 380, 390, ...",32.132097
5,9000,1,"[-22, -27, -33, -38, -40, -39, -36, -30, -23, ...",145.925780
6,9000,0,"[291, 345, 405, 465, 510, 527, 516, 509, 507, ...",16.995531


In [4]:
# Filter out rows with rr_std greater than 100
filtered_df = train_df[train_df["rr_std"] > 100]
#get total labels for each class
label_counts = filtered_df["label"].value_counts()
print("Label counts in filtered DataFrame:")
print(label_counts)
filtered_df

Label counts in filtered DataFrame:
label
0    1446
2     962
1     407
3     179
Name: count, dtype: int64


,length,label,signal,rr_std
1,18000,2,"[-446, -541, -637, -733, -819, -858, -867, -87...",182.922570
3,9000,0,"[-1255, -1488, -1745, -2015, -2253, -2374, -23...",123.855300
5,9000,1,"[-22, -27, -33, -38, -40, -39, -36, -30, -23, ...",145.925780
10,9000,0,"[43, 56, 63, 69, 73, 78, 82, 85, 88, 91, 93, 9...",154.062144
15,3178,1,"[107, 128, 155, 166, 173, 179, 183, 187, 190, ...",101.254042
...,...,...,...,...
6173,9000,1,"[-12, -14, -16, -17, -19, -21, -22, -35, -68, ...",133.101465
6174,9000,0,"[546, 588, 632, 679, 696, 673, 605, 536, 512, ...",223.473714
6175,9000,0,"[503, 600, 695, 789, 873, 915, 911, 891, 883, ...",115.361407
6177,9000,2,"[271, 469, 690, 862, 932, 875, 707, 456, 177, ...",112.098171


# MODEL 1

In [5]:

from torch.utils.data import DataLoader
set_seed(42)

# STDFT function to compute the Short-Time Fourier Transform (STFT) for a batch of signals
# It convert into time -frequency representation
# n_fft: size of the FFT window(300 hz is the sampling rate, so 256 is a good choice to capture more than 1 heart beat)
# hop_length: number of samples between successive frames (128 is a good choice since every window will overlap by 50%)
def compute_stft_batch(x, n_fft=256, hop_length=128):
    stft = torch.stft(
        x, n_fft=n_fft, hop_length=hop_length,
        return_complex=True
    )
    return torch.abs(stft)  

# 2. Model
class ECGModel(nn.Module):
    def __init__(self,hidden_size=128, dropout_rate=0.0):
        super(ECGModel, self).__init__()
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2))
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2))
        )

        # RNN
        self.rnn = nn.GRU(
            input_size=32 * 32, # out_channels * frequency_bins // 2 // 2 (due to max pooling) ()
            hidden_size= hidden_size,
            batch_first=True
        )

        # Fully connected
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, 4)  # 4 classes: Normal, AF, Other, Noisy

    def forward(self, x:torch.Tensor)-> torch.Tensor:
        # x: (batch_size, signal_length)
        #print("Input:", x.shape)
        x = compute_stft_batch(x)  # STFT → (batch, freq, time)
        #print("After STFT:", x.shape)
        x = torch.log1p(x)  # logarithmic scaling
        #print("After log1p:", x.shape)
        

        x = x.unsqueeze(1)  # CNN input shape: (batch, channel, freq, time)
        #print("After Unsqueeze:", x.shape)

        x = self.conv1(x)
        x = self.conv2(x)
        #print("After conv2:", x.shape)

        # Flatten the output for RNN input
        b, c, f, t = x.shape  # batch, channel, freq, time
        x = x.view(b, c * f, t)  # (batch, features, time)
        #print("After view:", x.shape)
        x = x.permute(0, 2, 1)   # (batch, time, features)
        #print("After permute:", x.shape)

        # RNN
        output, h_n = self.rnn(x)
        #print("After RNN output:", output.shape)
        x = self.dropout(h_n[-1])
        x = self.fc(x)  # use the last hidden state for classification
        #print("Final output:", x.shape)
        return x


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Augmentation
Number and type of augmentation specific to each class

Only 1 random augmentation per signal

Filtering based on the rr_std limit (only clean signals are augmented)

Noisy class is excluded

In [7]:
set_seed(42)
def pad_or_trim(signal, target_length=9000):
    current_length = len(signal)

    if current_length < target_length:
        # Pad with zeros at the end
        padding = target_length - current_length
        signal = np.pad(signal, (0, padding), 'constant')
    elif current_length > target_length:
        # Trim from center
        start = (current_length - target_length) // 2
        signal = signal[start : start + target_length]

    return signal

def time_shift(signal, shift_max=100):
    shift = np.random.randint(-shift_max, shift_max)
    return np.roll(signal, shift)


def scale_amplitude(signal, scale_range=(0.95, 1.05)):
    scale = np.random.uniform(*scale_range)
    return signal * scale

def add_noise(signal, noise_level=0.001):
    noise = np.random.normal(0, noise_level * np.std(signal), size=signal.shape)
    return signal + noise

In [8]:
set_seed(42)
# Augmentation strategy
augment_strategy = {
    0: 1,  # Normal
    1: 5,  # AF
    2: 2,  # Other
    3: 0   # Noisy
}

# Possible augmentation types for each class
augment_types_per_class = {
    0: ["noise", "shift", "scale"],      # Normal
    1: ["shift", "scale"],               # AF (noise yok!)
    2: ["noise", "shift", "scale"],      # Other
    3: []                                # Noisy (yok)
}

# Define the rr_std limits for each label for augmentation based ond EDA
# 0: Normal, 1: AF, 2: Other, 3: Noisy
rr_std_limits = {
    0: 110,  
    1: 150,  
    2: 130,  
    3: 0    
}


In [9]:
set_seed(42)
augmented_data = []

for i, row in train_df.iterrows():
    signal = row["signal"]
    label = row["label"]
    rr_std = row["rr_std"]

    # Normalize
    signal = pad_or_trim(signal)

    # check if augmentation is needed
    if rr_std < rr_std_limits[label] and augment_strategy[label] > 0:
        for _ in range(augment_strategy[label]):
            aug_signal = signal.copy()
            aug_signal = np.array(aug_signal)

            # choose a random augmentation type for the current label
            np.random.seed(42) 
            aug_type = np.random.choice(augment_types_per_class[label])

            if aug_type == "noise":
                aug_signal = add_noise(aug_signal)
            elif aug_type == "shift":
                aug_signal = time_shift(aug_signal)
            elif aug_type == "scale":
                aug_signal = scale_amplitude(aug_signal)

            aug_signal = pad_or_trim(aug_signal)

            augmented_data.append({
                "signal": aug_signal.tolist(),
                "label": label,
                "length": 9000,
                "rr_std": rr_std
            })



train_df["signal"] = train_df["signal"].apply(pad_or_trim)
train_df["length"] = 9000

full_train_df = pd.concat([train_df, pd.DataFrame(augmented_data)], ignore_index=True)

output_path = f"{DATA_DIR}/train_df_final_augmented.pkl"
full_train_df.to_pickle(output_path)
print(f"\n✅ Augment edilmiş veri kaydedildi: {len(full_train_df)} örnek -> {output_path}")


✅ Augment edilmiş veri kaydedildi: 9914 örnek -> C:\Users\emert\Desktop\ecg-timeseries-model\2_models/train_df_final_augmented.pkl


In [10]:
set_seed(42)
from torch.utils.data import DataLoader
def train_one_epoch(model:nn.Module, 
                    dataloader:DataLoader, 
                    optimizer:torch.optim.Optimizer, 
                    loss_fn:nn.Module):
    """
    Trains the model for one full epoch.

    Args:
        model: The neural network model (ECGModel).
        dataloader: The training DataLoader providing batches.
        optimizer: The optimizer (e.g., Adam).
        loss_fn: The loss function (e.g., CrossEntropyLoss).

    Returns:
        A tuple of (average_loss, accuracy) for the epoch.
    """
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for signals, labels in dataloader:
        signals = signals.to(device)
        labels = labels.to(device)

        outputs = model(signals)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * signals.size(0)

        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy


In [11]:
from sklearn.utils.class_weight import compute_class_weight

set_seed(42)

# Compute class weights based on the training labels
original_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array([0, 1, 2, 3]),
    y=full_train_df['label'].values
)

# Normalize and scale the weights
scaled_weights = original_weights / original_weights.max()  # normalize to max=1
scaled_weights = 0.5 + (scaled_weights * 0.5)  # shrink range to [0.5, 1.0] for balance

weights_tensor = torch.tensor(scaled_weights, dtype=torch.float32).to(device)

# Weighted loss function
loss_fn_weighted = nn.CrossEntropyLoss(weight=weights_tensor)

print("Original weights: ", original_weights)
print("Scaled weights: ", scaled_weights)


Original weights:  [ 0.51124175  1.42853026  0.78958267 12.90885417]
Scaled weights:  [0.51980198 0.55533141 0.53058299 1.        ]


In [12]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report, precision_score, recall_score,accuracy_score

def evaluate_with_metrics(model:nn.Module, dataloader, loss_fn, device):
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0

    with torch.no_grad():
        for signals, labels in dataloader:
            signals = signals.to(device)
            labels = labels.to(device)

            outputs = model(signals)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    cm = confusion_matrix(all_labels, all_preds, labels=[0, 1, 2, 3])
    report = classification_report(
    all_labels, all_preds,
    labels=[0, 1, 2, 3],
    target_names=['Normal', 'AF', 'Other', 'Noisy'],
    zero_division=0  # uyarı vermesin
)

    return avg_loss, accuracy, f1, precision, cm, report


In [13]:
from torch.utils.data import Dataset


class ECGDataset(Dataset):
    def __init__(self, df, target_length=9000):
        self.df = df
        self.target_length = target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Signal processing
        # Pad or trim the signal to the target length
        signal = pad_or_trim(row['signal'], self.target_length)
        signal = torch.tensor(signal, dtype=torch.float32)
        label = int(row['label']) 

        return signal, label

In [14]:
print(" Full train DataFrame shape:", full_train_df.shape)
print(" Validation DataFrame shape:", val_df.shape)
print(full_train_df["label"].value_counts().sort_index())

 Full train DataFrame shape: (9914, 4)
 Validation DataFrame shape: (927, 4)
label
0    4848
1    1735
2    3139
3     192
Name: count, dtype: int64


In [15]:
set_seed(42)
train_dataset = ECGDataset(full_train_df)
val_dataset = ECGDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, generator=torch.Generator().manual_seed(42))
val_loader = DataLoader(val_dataset, batch_size=32)

model = ECGModel(hidden_size=256, dropout_rate=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.0005)
loss_fn = nn.CrossEntropyLoss(weight=weights_tensor.to(device))


c:\Users\emert\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
import copy


num_epochs = 20
best_val_f1 = 0
patience = 5
counter = 0
best_model_state = None
for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, loss_fn)
    val_loss, val_acc, val_f1, val_precision, cm, val_report = evaluate_with_metrics(model, val_loader, loss_fn, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")
    print(f"Val   F1: {val_f1:.4f} | Val Precision: {val_precision:.4f}")
    print("-" * 40)
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_model_state =  copy.deepcopy(model.state_dict())
        counter = 0
    else:
        counter += 1
        print(f"EarlyStopping Counter: {counter} / {patience}")
        if counter >= patience:
            print("🔴 Early stopping triggered!")
            break

c:\Users\emert\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/20
Train Loss: 1.1250 | Train Acc: 0.4815
Val   Loss: 0.9671 | Val   Acc: 0.5804
Val   F1: 0.2165 | Val Precision: 0.3989
----------------------------------------
Epoch 2/20
Train Loss: 1.0054 | Train Acc: 0.5533
Val   Loss: 0.9232 | Val   Acc: 0.5944
Val   F1: 0.3577 | Val Precision: 0.3678
----------------------------------------
Epoch 3/20
Train Loss: 0.8545 | Train Acc: 0.6383
Val   Loss: 0.8452 | Val   Acc: 0.6408
Val   F1: 0.3735 | Val Precision: 0.5221
----------------------------------------
Epoch 4/20
Train Loss: 0.7942 | Train Acc: 0.6597
Val   Loss: 0.8941 | Val   Acc: 0.6084
Val   F1: 0.4528 | Val Precision: 0.6000
----------------------------------------
Epoch 5/20
Train Loss: 0.7584 | Train Acc: 0.6739
Val   Loss: 0.8317 | Val   Acc: 0.6472
Val   F1: 0.4648 | Val Precision: 0.5878
----------------------------------------
Epoch 6/20
Train Loss: 0.7344 | Train Acc: 0.6845
Val   Loss: 0.8352 | Val   Acc: 0.6365
Val   F1: 0.4631 | Val Precision: 0.5191
---------------

In [17]:

model.load_state_dict(best_model_state)

<All keys matched successfully>

In [18]:
val_loss, val_acc, val_f1, val_precision, val_cm, val_report = evaluate_with_metrics(model, val_loader, loss_fn, device)
print(f"Validation F1 Score: {val_f1:.4f}")
print("Confusion Matrix:\n", val_cm)
print("Classification Report:\n", val_report)
print("vall acc:", val_acc)

Validation F1 Score: 0.6195
Confusion Matrix:
 [[464   3  76   1]
 [ 23  38  21   2]
 [121  15 127   1]
 [ 10   3   4  18]]
Classification Report:
               precision    recall  f1-score   support

      Normal       0.75      0.85      0.80       544
          AF       0.64      0.45      0.53        84
       Other       0.56      0.48      0.52       264
       Noisy       0.82      0.51      0.63        35

    accuracy                           0.70       927
   macro avg       0.69      0.58      0.62       927
weighted avg       0.69      0.70      0.69       927

vall acc: 0.697950377562028
